# Data Cleaning 

In [7]:
# necessary to import db_connector script
import sys
import os

# Get the absolute path of the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add project root to sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [8]:
import pandas as pd
import numpy as np 
import datetime 
import re 

from db_connector import load_from_excel

In [9]:
data = load_from_excel()

# Make copies to avoid modifying originals
cleaned_data = {key: df.copy() for key, df in data.items()}

print("Data loaded successfully.")

Data loaded successfully.


## Clean Requisitions Table

In [18]:
print("\n=== Cleaning Requisitions Table ===")

# 1. Handling missing values 
# a. Fix RECRUITER field (30.6% missing)

print("\n1. Handling missing values:")
missing_recruiter = cleaned_data['requisitions']['RECRUITER'].isnull().sum()
print(f"   - Missing RECRUITER values: {missing_recruiter} ({missing_recruiter/len(cleaned_data['requisitions'])*100:.2f}%)")

# Use RECRUITER_ID to fill in missing RECRUITER values where possible
recruiter_map = cleaned_data['requisitions'][cleaned_data['requisitions']['RECRUITER'].notna()].groupby('RECRUITER_ID')['RECRUITER'].first().to_dict()
print(f"   - Found {len(recruiter_map)} unique RECRUITER_ID to RECRUITER mappings")

# Fill missing values using the mapping
before_fill = cleaned_data['requisitions']['RECRUITER'].isnull().sum()
cleaned_data['requisitions']['RECRUITER'] = cleaned_data['requisitions'].apply(
    lambda row: recruiter_map.get(row['RECRUITER_ID']) if pd.isnull(row['RECRUITER']) and row['RECRUITER_ID'] in recruiter_map else row['RECRUITER'], 
    axis=1
)
after_fill = cleaned_data['requisitions']['RECRUITER'].isnull().sum()
print(f"   - Filled {before_fill - after_fill} missing RECRUITER values")
print(f"   - Remaining missing RECRUITER values: {after_fill} ({after_fill/len(cleaned_data['requisitions'])*100:.2f}%)")

# b.  Handle missing CLOSE_DATE (5.9% missing) - these are likely still open
open_reqs = cleaned_data['requisitions'][cleaned_data['requisitions']['CLOSE_DATE'].isnull()]
print(f"   - Found {len(open_reqs)} requisitions with missing CLOSE_DATE (likely still open)")

# Check if these align with STATUS_IN
if 'STATUS_IN' in cleaned_data['requisitions'].columns:
    still_open = open_reqs[open_reqs['STATUS_IN'] == 'Open']
    incorrectly_marked = open_reqs[open_reqs['STATUS_IN'] != 'Open']
    
    print(f"   - {len(still_open)} of these are correctly marked as 'Open' in STATUS_IN")
    if len(incorrectly_marked) > 0:
        print(f"   - {len(incorrectly_marked)} have missing CLOSE_DATE but aren't marked as 'Open'")

        cleaned_data['requisitions'].loc[incorrectly_marked.index, 'STATUS_IN'] = 'Open'

# 2. Check for unusual NUMBER_OF_OPENINGS
high_openings = cleaned_data['requisitions'][cleaned_data['requisitions']['NUMBER_OF_OPENINGS'] > 10]
print(f"\n2. Found {len(high_openings)} requisitions with more than 10 openings")

# 3. Handle invalid dates and time-to-fill outliers
# a. Flag requisitions with close date before open date
invalid_dates = cleaned_data['requisitions'][
    cleaned_data['requisitions']['CLOSE_DATE'].notna() & 
    (cleaned_data['requisitions']['CLOSE_DATE'] < cleaned_data['requisitions']['OPEN_DATE'])
]
cleaned_data['requisitions']['INVALID_DATES_FLAG'] = False
cleaned_data['requisitions'].loc[invalid_dates.index, 'INVALID_DATES_FLAG'] = True
print(f"\n3. Found {len(invalid_dates)} requisitions with close date before open date")
print("   - These have been flagged with INVALID_DATES_FLAG")

# b. Calculate and flag time-to-fill outliers
closed_reqs = cleaned_data['requisitions'][cleaned_data['requisitions']['CLOSE_DATE'].notna()].copy()
if not closed_reqs.empty:
    closed_reqs['time_to_fill'] = (closed_reqs['CLOSE_DATE'] - closed_reqs['OPEN_DATE']).dt.days
    
    # Calculate outlier bounds
    q1 = closed_reqs['time_to_fill'].quantile(0.25)
    q3 = closed_reqs['time_to_fill'].quantile(0.75)
    iqr = q3 - q1
    lower_bound = max(0, q1 - 1.5 * iqr)
    upper_bound = q3 + 1.5 * iqr
    
    # Add time_to_fill and outlier flag columns
    cleaned_data['requisitions']['time_to_fill'] = closed_reqs['time_to_fill']
    cleaned_data['requisitions']['TIME_TO_FILL_OUTLIER'] = False
    cleaned_data['requisitions'].loc[
        closed_reqs[
            (closed_reqs['time_to_fill'] < lower_bound) | 
            (closed_reqs['time_to_fill'] > upper_bound)
        ].index, 
        'TIME_TO_FILL_OUTLIER'
    ] = True
    
    print(f"   - Added time_to_fill calculation and flagged {cleaned_data['requisitions']['TIME_TO_FILL_OUTLIER'].sum()} outliers") 

# 4. Create a cleaned version ready for analysis
cleaned_requisitions = cleaned_data['requisitions'].copy()
print("\nRequisitions cleaning completed.")
print(f"Original shape: {data['requisitions'].shape}, Cleaned shape: {cleaned_requisitions.shape}")


=== Cleaning Requisitions Table ===

1. Handling missing values:
   - Missing RECRUITER values: 0 (0.00%)
   - Found 37 unique RECRUITER_ID to RECRUITER mappings
   - Filled 0 missing RECRUITER values
   - Remaining missing RECRUITER values: 0 (0.00%)
   - Found 296 requisitions with missing CLOSE_DATE (likely still open)
   - 296 of these are correctly marked as 'Open' in STATUS_IN

2. Found 25 requisitions with more than 10 openings

3. Found 0 requisitions with close date before open date
   - These have been flagged with INVALID_DATES_FLAG
   - Added time_to_fill calculation and flagged 197 outliers

Requisitions cleaning completed.
Original shape: (5025, 11), Cleaned shape: (5025, 15)


## Clean Candidate Dataset

In [ ]:
print("\n=== Cleaning Candidate Dataset ===")

# 1. Handle duplicates
duplicate_count = cleaned_data['candidate'].duplicated().sum()
print(f"1. Found {duplicate_count} duplicate records ({duplicate_count/len(cleaned_data['candidate'])*100:.2f}%)")
cleaned_data['candidate'] = cleaned_data['candidate'].drop_duplicates()
print(f"   - Removed {duplicate_count} duplicate records")

# 2. Fix date columns
date_cols = ['SUBMISSION_DATE', 'HISTORICAL_STATUS_START_DATE', 'HISTORICAL_STATUS_END_DATE', 'LAST_MODIFIED_DATE']
for col in date_cols:
    # Convert to datetime, set errors to coerce to handle invalid dates
    if cleaned_data['candidate'][col].dtype != 'datetime64[ns]':
        cleaned_data['candidate'][col] = pd.to_datetime(cleaned_data['candidate'][col], errors='coerce')
        print(f"   - Converted {col} to datetime format")

# 3. Handle records with illogical date sequences
# Check submission_date to historical_status_start_date
illogical_dates = cleaned_data['candidate'][
    (cleaned_data['candidate']['SUBMISSION_DATE'].notna()) & 
    (cleaned_data['candidate']['HISTORICAL_STATUS_START_DATE'].notna()) & 
    (cleaned_data['candidate']['HISTORICAL_STATUS_START_DATE'] < cleaned_data['candidate']['SUBMISSION_DATE'])
]
print(f"\n3. Found {len(illogical_dates)} records where status start date is before submission date")
print("   - These will be flagged but kept in the dataset")
cleaned_data['candidate']['ILLOGICAL_DATE_FLAG'] = False
cleaned_data['candidate'].loc[illogical_dates.index, 'ILLOGICAL_DATE_FLAG'] = True

# 4. Handle missing CANDIDATE_ID values
missing_ids = cleaned_data['candidate'][cleaned_data['candidate']['CANDIDATE_ID'].isna()]
print(f"\n4. Found {len(missing_ids)} records with missing CANDIDATE_ID")
print("   - These will be flagged but kept in the dataset")
cleaned_data['candidate']['MISSING_ID_FLAG'] = cleaned_data['candidate']['CANDIDATE_ID'].isna()

# 5. Handle invalid status values
if 'candidate_status' in data:
    valid_statuses = set(data['candidate_status']['CANDIDATE_HISTORICAL_STATUS'])
    invalid_statuses = cleaned_data['candidate'][~cleaned_data['candidate']['CANDIDATE_HISTORICAL_STATUS'].isin(valid_statuses)]
    cleaned_data['candidate']['INVALID_STATUS_FLAG'] = ~cleaned_data['candidate']['CANDIDATE_HISTORICAL_STATUS'].isin(valid_statuses)
    print(f"\n5. Found {len(invalid_statuses)} records with invalid status values")
    print("   - These have been flagged with INVALID_STATUS_FLAG")

# 6. Handle pipeline timing outliers
# a. Submission to interview time
sub_to_int = cleaned_data['candidate'][
    (cleaned_data['candidate']['SUBMISSION_DATE'].notna()) & 
    (cleaned_data['candidate']['INTERVIEW_DATE'].notna())
].copy()

if not sub_to_int.empty:
    sub_to_int['days_to_interview'] = (sub_to_int['INTERVIEW_DATE'] - sub_to_int['SUBMISSION_DATE']).dt.days
    sub_to_int = sub_to_int[sub_to_int['days_to_interview'] >= 0]
    
    q1 = sub_to_int['days_to_interview'].quantile(0.25)
    q3 = sub_to_int['days_to_interview'].quantile(0.75)
    iqr = q3 - q1
    upper_bound = q3 + 1.5 * iqr
    
    cleaned_data['candidate']['days_to_interview'] = sub_to_int['days_to_interview']
    cleaned_data['candidate']['INTERVIEW_TIMING_OUTLIER'] = False
    cleaned_data['candidate'].loc[
        sub_to_int[sub_to_int['days_to_interview'] > upper_bound].index,
        'INTERVIEW_TIMING_OUTLIER'
    ] = True
    
    print("\n6. Pipeline timing outliers:")
    print(f"   - Flagged {cleaned_data['candidate']['INTERVIEW_TIMING_OUTLIER'].sum()} interview timing outliers")

# b. Interview to hire time
int_to_hire = cleaned_data['candidate'][
    (cleaned_data['candidate']['INTERVIEW_DATE'].notna()) & 
    (cleaned_data['candidate']['HIRED_DATE'].notna())
].copy()

if not int_to_hire.empty:
    int_to_hire['days_to_hire'] = (int_to_hire['HIRED_DATE'] - int_to_hire['INTERVIEW_DATE']).dt.days
    int_to_hire = int_to_hire[int_to_hire['days_to_hire'] >= 0]
    
    q1 = int_to_hire['days_to_hire'].quantile(0.25)
    q3 = int_to_hire['days_to_hire'].quantile(0.75)
    iqr = q3 - q1
    upper_bound = q3 + 1.5 * iqr
    
    cleaned_data['candidate']['days_to_hire'] = int_to_hire['days_to_hire']
    cleaned_data['candidate']['HIRE_TIMING_OUTLIER'] = False
    cleaned_data['candidate'].loc[
        int_to_hire[int_to_hire['days_to_hire'] > upper_bound].index,
        'HIRE_TIMING_OUTLIER'
    ] = True
    
    print(f"   - Flagged {cleaned_data['candidate']['HIRE_TIMING_OUTLIER'].sum()} hire timing outliers")

# 7. Check and flag status progression logic
# Define expected status progression
status_order = {
    'Applied': 1,
    'Screening': 2,
    'Interview': 3,
    'Offer': 4,
    'Hired': 5,
    'Rejected': -1,  # Can happen at any stage
    'Withdrawn': -1  # Can happen at any stage
}

# Create a status progression check
def check_status_progression(group):
    # Sort by status start date
    sorted_statuses = group.sort_values('HISTORICAL_STATUS_START_DATE')
    
    prev_stage = 0
    valid_progression = True
    
    for status in sorted_statuses['CANDIDATE_HISTORICAL_STATUS']:
        if status not in status_order:
            continue
            
        current_stage = status_order[status]
        
        # Skip progression check for rejected/withdrawn
        if current_stage == -1:
            continue
            
        # Check if current stage is valid progression
        if current_stage < prev_stage:
            valid_progression = False
            break
            
        prev_stage = current_stage
    
    return valid_progression

# Group by candidate and check progression
progression_results = cleaned_data['candidate'].groupby('CANDIDATE_ID').apply(check_status_progression)

# Add flag for invalid progressions
cleaned_data['candidate']['INVALID_PROGRESSION_FLAG'] = False
for candidate_id, is_valid in progression_results.items():
    if not is_valid:
        cleaned_data['candidate'].loc[
            cleaned_data['candidate']['CANDIDATE_ID'] == candidate_id, 
            'INVALID_PROGRESSION_FLAG'
        ] = True

invalid_count = cleaned_data['candidate']['INVALID_PROGRESSION_FLAG'].sum()
print(f"\n5. Found {invalid_count} candidates with invalid status progressions")
print("   - These have been flagged with INVALID_PROGRESSION_FLAG")


# 8. Create a clean version ready for analysis
cleaned_candidate = cleaned_data['candidate'].copy()
print("\nCandidate cleaning completed.")
print(f"Original shape: {data['candidate'].shape}, Cleaned shape: {cleaned_candidate.shape}")



=== Cleaning Candidate Dataset ===
1. Found 0 duplicate records (0.00%)
   - Removed 0 duplicate records
   - Converted SUBMISSION_DATE to datetime format

3. Found 5723 records where status start date is before submission date
   - These will be flagged but kept in the dataset

4. Found 17 records with missing CANDIDATE_ID
   - These will be flagged but kept in the dataset

Candidate cleaning completed.
Original shape: (615707, 9), Cleaned shape: (615707, 11)


## Clean Candidate Status Table

In [12]:
# 1. Remove duplicates
duplicate_count = cleaned_data['candidate_status'].duplicated().sum()
print(f"\n1. Found {duplicate_count} duplicate records")
cleaned_data['candidate_status'] = cleaned_data['candidate_status'].drop_duplicates()
print(f"   - Removed {duplicate_count} duplicate records")

# 3. Create a clean version ready for analysis
cleaned_candidate_status = cleaned_data['candidate_status'].copy()
print("\nCandidate Status cleaning completed.")
print(f"Original shape: {data['candidate_status'].shape}, Cleaned shape: {cleaned_candidate_status.shape}")


1. Found 0 duplicate records
   - Removed 0 duplicate records

Candidate Status cleaning completed.
Original shape: (16, 2), Cleaned shape: (16, 2)


In [13]:
# Save cleaned data to cleaned_data dictionary
cleaned_data = {
    'requisitions': cleaned_requisitions,
    'candidate': cleaned_candidate,
    'candidate_status': cleaned_candidate_status,
    'department': cleaned_data['department']
}

print("\n=== Summary of Cleaning Results ===")
for table_name, df in cleaned_data.items():
    original_shape = data[table_name].shape
    cleaned_shape = df.shape
    print(f"{table_name}: Original {original_shape} -> Cleaned {cleaned_shape}")

print("\nCleaning process completed successfully!")


=== Summary of Cleaning Results ===
requisitions: Original (5025, 11) -> Cleaned (5025, 12)
candidate: Original (615707, 9) -> Cleaned (615707, 11)
candidate_status: Original (16, 2) -> Cleaned (16, 2)
department: Original (392, 4) -> Cleaned (392, 4)

Cleaning process completed successfully!
